# Project 3: Classification of Subreddits

## Problem Statement

People seek advice from each other all the time, especially on social media platforms like **Reddit** (where they can maintain their anonymity). Given a post from a user seeking advice on a subreddit, is it possible to **categorize the post into different categories (by employing different Machine Learning techniques)**? If yes, **how accurately can this advice request categorization be done?**

## Executive Summary

According to this [page](https://foundationinc.co/lab/reddit-statistics/), Reddit is a rapidly growing social media sharing platform with a 330 million active users count as of 2018, and a 30% increase in the active users count in 2019. People come to Reddit for all different types of reasons, ranging from just browsing the latest [meme content](https://www.reddit.com/r/funny/) or news to connecting with people of similar interests, sharing their experiences, etc. **One important reason for people to visit Reddit is to seek and give advice (on a broad range of topics), or just about their situations/experiences in life in general.**

Reddit is trying to introduce **a new feature to automatically suggest categories/subreddits to users for posting their requests for advice**, based on the text of the user's post seeking the advice. The feature is being introduced to better guide people seeking advice to the appropriate subreddits, so that their requests can be answered in a better and effective manner by the dedicated subreddit community. To introduce this feature, Reddit is trying to understand whether or not it is possible to automatically categorize the advice posts based on their text. And if it is indeed possible to categorize such posts, how effectively/accurately can this done?

To answer this question and give Reddit a brief sample of the possibilities of machine learning, in this project, I seek to apply different data classification techniques to classify textual posts from **two subreddits - [r/relationship_advice](https://www.reddit.com/r/relationship_advice/) (dedicated for people seeking advice on their personal relationships) and [r/legaladvice](https://www.reddit.com/r/legaladvice/) (dedicated for people seeking semi-professional/professional advice on legal issues)**.

For ease of organization and understanding, the project has been **divided into 2 separate Jupyter notebooks**. The first notebook focuses on data gathering via web-scraping from the Reddit API. In order to do the classification modelling, about a 1000 posts were scraped from each of the two subreddits (sorted by newest at the time of scraping). The second notebook then focuses on data cleaning, EDA and different techniques of classification modelling.

In order to classify the posts from the two subreddits, I employ different methods of classification, such as **Logistic Regression, K-Nearest Neighbors classifier, Naive Bayes classifier and Random Forests**. Each of the models and their hyperparameters are optimized by passing them through **Cross-Validated Grid Searching (GridSearchCV)**. As I am dealing with textual data, I also employ different **NLP techniques** to convert text into machine-understandable forms of data, such as **Count Vectorization and TF-IDF Vectorization**. Each of the classification models built are assessed for generalizability using the **Accuracy metric**, with the final selected model being evaluated on further metrics such as **Sensitivity (True Positive Rate), Specificity (True Negative Rate) and ROC AUC**.

### Contents:
- Jupyter Notebook 1 - ***1_data_gathering_from_reddit_api.ipynb***
    - [Data Gathering](#Data-Gathering)
        - [Web Scraping via Reddit API](#Web-Scraping-via-Reddit-API)
        - [Verifying the Scraped Data](#Verifying-the-Scraped-Data)
- Jupyter Notebook 2 - ***2_pre_processing_classification_evaluation.ipynb***
    - Data Importing and Cleaning
        - Data Importing
        - Data Cleaning and Pre-processing
    - Exploratory Data Analysis](#Exploratory-Data-Analysis)
        - Visualizing Differences in User Activity Levels
        - Visualizing Differences in Posts
    - Classification Modelling](#Classification-Modelling)
        - Model Preparation
        - Classification Metrics
        - Vectorization of Words (CountVectorizer & TfidfVectorizer with LogisticRegression)
        - TfidfVectorizer with KNeighborsClassifier
        - TfidfVectorizer with MultinomialNB
        - TfidfVectorizer with RandomForestClassifier
    - Final Classification Modelling and Evaluation
        - ROC Curve
        - Feature Importance
        - Evaluation Metrics Summary
        - Analysis of Misclassified Posts
    - Conclusions and Recommendations

In [1]:
# Imports

import requests
import pandas as pd
import time
import random

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

## Data Gathering

### Web Scraping via Reddit API

In [2]:
# Setting the URL of the subreddits to scrape posts in .json format.

legaladvice_url = 'https://www.reddit.com/r/legaladvice/new.json'
relationshipadvice_url = 'https://www.reddit.com/r/relationship_advice/new.json'

In [3]:
# Creating a dictionary of base URLs.

base_urls = {'legaladvice' : legaladvice_url, 'relationshipadvice' : relationshipadvice_url}

In [4]:
# Setting the query parameters.

params = {'limit' : 100, 'after' : None}

In [5]:
# Setting a custom header so as to not get the scraping attempt blocked by Reddit.

headers = {'User-agent': 'agent j. bourne'}

In [6]:
# Looping through the base URLs of the two subreddits to scrape 1000 posts from each subreddit.

all_posts = {}

# Looping through dictionary of subreddit names and URLs.
for subred, url in base_urls.items():
    
    # Printing current subreddit being scraped.
    print(f'\nSCRAPING SUBREDDIT: {subred}')
    
    # Reseting 'after' parameter to be None before first scraping attempt.
    # This parameter helps to keep track of last post from each successful scrape attempt,
    # so that next scrape is from the previous scrape's last post onwards.
    params['after'] = None
    
    # Initiating a new list for the current subreddit being scraped.
    all_posts[subred] = []
    
    # Counter to keep track of no. of total posts scraped so far. This will be used in the while loop below.
    posts_scraped = 0
    
    # While loop to scrape posts from current subreddit.
    while posts_scraped < 1000:
        
        # Requesting data from reddit API with custom user-agent in headers, and 'limit'=100 and 'after' as parameters.
        response = requests.get(url, headers=headers, params=params)
        
        # Checking for successful response from API.
        if response.status_code == 200:
            print("\nSuccessful scraping attempt.")
        else:
            print(f"\nUnsuccessful scraping attempt. Error status code: {response.status_code}")
            break
        
        response_dict = response.json()
        
        # Preparing list of all posts scraped in this attempt using list comprehension.
        current_posts = [posts['data'] for posts in response_dict['data']['children']]
        
        # Updating counter of no. of total posts scraped so far.
        # Printing relevant info messages about no. of posts scraped.
        posts_scraped += len(current_posts)
        print(f'Posts scraped in this attempt = {len(current_posts)}')
        print(f'Total posts scraped so far    = {posts_scraped}')
        
        # Updating 'after' parameter with the tag of last post scraped in this attempt,
        # so that next scrape attempt is from this post onwards.
        params['after'] = response_dict['data']['after']
        
        # Extending the list of posts scraped so far from current subreddit by adding posts scraped in this attempt.
        all_posts[subred].extend(current_posts)
        
        # Saving posts scraped so far as a CSV file (overwriting any previously written CSV file).
        pd.DataFrame(all_posts[subred]).to_csv(f'../data/{subred}_raw_posts.csv', mode='w', header=True, index=False)
        print(f'Saved posts to CSV: {subred}_raw_posts.csv')
        
        # Setting a random sleep timer of 5-15 seconds so as to not bombard reddit server with multiple requests within seconds.
        sleep_duration = random.randint(5,15)
        print(f'Sleeping for {sleep_duration} seconds...')
        time.sleep(sleep_duration)
    
print('\nDone.')


SCRAPING SUBREDDIT: legaladvice

Successful scraping attempt.
Posts scraped in this attempt = 100
Total posts scraped so far    = 100
Saved posts to CSV: legaladvice_raw_posts.csv
Sleeping for 11 seconds...

Successful scraping attempt.
Posts scraped in this attempt = 100
Total posts scraped so far    = 200
Saved posts to CSV: legaladvice_raw_posts.csv
Sleeping for 11 seconds...

Successful scraping attempt.
Posts scraped in this attempt = 100
Total posts scraped so far    = 300
Saved posts to CSV: legaladvice_raw_posts.csv
Sleeping for 5 seconds...

Successful scraping attempt.
Posts scraped in this attempt = 100
Total posts scraped so far    = 400
Saved posts to CSV: legaladvice_raw_posts.csv
Sleeping for 6 seconds...

Successful scraping attempt.
Posts scraped in this attempt = 100
Total posts scraped so far    = 500
Saved posts to CSV: legaladvice_raw_posts.csv
Sleeping for 15 seconds...

Successful scraping attempt.
Posts scraped in this attempt = 100
Total posts scraped so far  

In [7]:
# Checking 'all_posts' dictionary to ensure the desired subreddits and posts are correctly scraped.

all_posts.keys()

dict_keys(['legaladvice', 'relationshipadvice'])

### Verifying the Scraped Data

**Checking posts scraped from subreddit *r/legaladvice***

Number of posts in *all_posts* dictionary must be same as in *legaladvice_raw_posts.csv*

In [8]:
len(all_posts['legaladvice'])

1097

In [13]:
# Converting to dataframe

legaladvice_df = pd.DataFrame(all_posts['legaladvice'])

In [14]:
legaladvice_df.head()

,approved_at_utc,subreddit,selftext,author_fullname,saved,mod_reason_title,gilded,clicked,title,link_flair_richtext,subreddit_name_prefixed,hidden,pwls,link_flair_css_class,downs,hide_score,name,quarantine,link_flair_text_color,upvote_ratio,author_flair_background_color,subreddit_type,ups,total_awards_received,media_embed,author_flair_template_id,is_original_content,user_reports,secure_media,is_reddit_media_domain,is_meta,category,secure_media_embed,link_flair_text,can_mod_post,score,approved_by,author_premium,thumbnail,edited,author_flair_css_class,author_flair_richtext,gildings,content_categories,is_self,mod_note,created,link_flair_type,wls,removed_by_category,banned_by,author_flair_type,domain,allow_live_comments,selftext_html,likes,suggested_sort,banned_at_utc,view_count,archived,no_follow,is_crosspostable,pinned,over_18,all_awardings,awarders,media_only,can_gild,spoiler,locked,author_flair_text,treatment_tags,visited,removed_by,num_reports,distinguished,subreddit_id,mod_reason_by,removal_reason,link_flair_background_color,id,is_robot_indexable,report_reasons,author,discussion_type,num_comments,send_replies,whitelist_status,contest_mode,mod_reports,author_patreon_flair,author_flair_text_color,permalink,parent_whitelist_status,stickied,url,subreddit_subscribers,created_utc,num_crossposts,media,is_video,link_flair_template_id
0,None,legaladvice,I (19M) was with my abusive ex for about 5 mon...,t2_66b703t5,False,None,0,False,"(AZ, USA) My abusive ex destroyed my stuff",[],r/legaladvice,False,6,None,0,True,t3_ggjbe9,False,dark,1.0,None,public,1,0,{},None,False,[],None,False,False,None,{},None,False,1,None,False,,False,None,[],{},None,True,None,1.589073e+09,text,6,None,None,text,self.legaladvice,False,"&lt;!-- SC_OFF --&gt;&lt;div class=""md""&gt;&lt...",None,None,None,None,False,True,False,False,False,[],[],False,False,False,False,None,[],False,None,None,None,t5_2rawz,None,None,,ggjbe9,True,None,Independent_Scale,None,0,True,all_ads,False,[],False,None,/r/legaladvice/comments/ggjbe9/az_usa_my_abusi...,all_ads,False,https://www.reddit.com/r/legaladvice/comments/...,1237033,1.589044e+09,0,None,False,NaN
1,None,legaladvice,Our backyard is small only 20 ft wide. Our nei...,t2_5z5khlwm,False,None,0,False,Neighbors encroached 1.5ft with new femce and ...,[],r/legaladvice,False,6,None,0,True,t3_ggj9vg,False,dark,1.0,None,public,1,0,{},None,False,[],None,False,False,None,{},None,False,1,None,False,,False,None,[],{},None,True,None,1.589073e+09,text,6,None,None,text,self.legaladvice,False,"&lt;!-- SC_OFF --&gt;&lt;div class=""md""&gt;&lt...",None,None,None,None,False,True,False,False,False,[],[],False,False,False,False,None,[],False,None,None,None,t5_2rawz,None,None,,ggj9vg,True,None,1he_8igger_1h3_Beter,None,0,True,all_ads,False,[],False,None,/r/legaladvice/comments/ggj9vg/neighbors_encro...,all_ads,False,https://www.reddit.com/r/legaladvice/comments/...,1237033,1.589044e+09,0,None,False,NaN
2,None,legaladvice,"Hi, I am a worker who was temporarily furlough...",t2_55j1c6ak,False,None,0,False,[MN] Would I still receive unemployment benefi...,[],r/legaladvice,False,6,None,0,True,t3_ggj838,False,dark,1.0,None,public,1,0,{},None,False,[],None,False,False,None,{},None,False,1,None,False,,False,None,[],{},None,True,None,1.589073e+09,text,6,None,None,text,self.legaladvice,False,"&lt;!-- SC_OFF --&gt;&lt;div class=""md""&gt;&lt...",None,None,None,None,False,True,False,False,False,[],[],False,False,False,False,None,[],False,None,None,None,t5_2rawz,None,None,,ggj838,True,None,AlternatorDrounds,None,1,True,all_ads,False,[],False,None,/r/legaladvice/comments/ggj838/mn_would_i_stil...,all_ads,False,https://www.reddit.com/r/legaladvice/comments/...,1237033,1.589044e+09,0,None,False,NaN
3,None,legaladvice,I live in Belgium with my 2 daughters and my p...,t2_5ug9l5mp,False,None,0,False,My baby’s dad is kicking me out of the house b...,[],r/legaladvice,False,6,None,0,True,t3_ggj52z,False,dark,1.0,None,public,1,0,{},None,False,[],None,False,False,None,{

In [15]:
legaladvice_df.shape

(1097, 102)

In [18]:
# Checking for unique posts in total scraped posts.

len(legaladvice_df['name'].unique())

998

In [24]:
# Reading the CSV file as dataframe to ensure all data got saved correctly.

legaladvice_from_csv = pd.read_csv('../data/legaladvice_raw_posts.csv')

In [25]:
legaladvice_from_csv.head()

,approved_at_utc,subreddit,selftext,author_fullname,saved,mod_reason_title,gilded,clicked,title,link_flair_richtext,subreddit_name_prefixed,hidden,pwls,link_flair_css_class,downs,hide_score,name,quarantine,link_flair_text_color,upvote_ratio,author_flair_background_color,subreddit_type,ups,total_awards_received,media_embed,author_flair_template_id,is_original_content,user_reports,secure_media,is_reddit_media_domain,is_meta,category,secure_media_embed,link_flair_text,can_mod_post,score,approved_by,author_premium,thumbnail,edited,author_flair_css_class,author_flair_richtext,gildings,content_categories,is_self,mod_note,created,link_flair_type,wls,removed_by_category,banned_by,author_flair_type,domain,allow_live_comments,selftext_html,likes,suggested_sort,banned_at_utc,view_count,archived,no_follow,is_crosspostable,pinned,over_18,all_awardings,awarders,media_only,can_gild,spoiler,locked,author_flair_text,treatment_tags,visited,removed_by,num_reports,distinguished,subreddit_id,mod_reason_by,removal_reason,link_flair_background_color,id,is_robot_indexable,report_reasons,author,discussion_type,num_comments,send_replies,whitelist_status,contest_mode,mod_reports,author_patreon_flair,author_flair_text_color,permalink,parent_whitelist_status,stickied,url,subreddit_subscribers,created_utc,num_crossposts,media,is_video,link_flair_template_id
0,NaN,legaladvice,I (19M) was with my abusive ex for about 5 mon...,t2_66b703t5,False,NaN,0,False,"(AZ, USA) My abusive ex destroyed my stuff",[],r/legaladvice,False,6,NaN,0,True,t3_ggjbe9,False,dark,1.0,NaN,public,1,0,{},NaN,False,[],NaN,False,False,NaN,{},NaN,False,1,NaN,False,NaN,False,NaN,[],{},NaN,True,NaN,1.589073e+09,text,6,NaN,NaN,text,self.legaladvice,False,"&lt;!-- SC_OFF --&gt;&lt;div class=""md""&gt;&lt...",NaN,NaN,NaN,NaN,False,True,False,False,False,[],[],False,False,False,False,NaN,[],False,NaN,NaN,NaN,t5_2rawz,NaN,NaN,NaN,ggjbe9,True,NaN,Independent_Scale,NaN,0,True,all_ads,False,[],False,NaN,/r/legaladvice/comments/ggjbe9/az_usa_my_abusi...,all_ads,False,https://www.reddit.com/r/legaladvice/comments/...,1237033,1.589044e+09,0,NaN,False,NaN
1,NaN,legaladvice,Our backyard is small only 20 ft wide. Our nei...,t2_5z5khlwm,False,NaN,0,False,Neighbors encroached 1.5ft with new femce and ...,[],r/legaladvice,False,6,NaN,0,True,t3_ggj9vg,False,dark,1.0,NaN,public,1,0,{},NaN,False,[],NaN,False,False,NaN,{},NaN,False,1,NaN,False,NaN,False,NaN,[],{},NaN,True,NaN,1.589073e+09,text,6,NaN,NaN,text,self.legaladvice,False,"&lt;!-- SC_OFF --&gt;&lt;div class=""md""&gt;&lt...",NaN,NaN,NaN,NaN,False,True,False,False,False,[],[],False,False,False,False,NaN,[],False,NaN,NaN,NaN,t5_2rawz,NaN,NaN,NaN,ggj9vg,True,NaN,1he_8igger_1h3_Beter,NaN,0,True,all_ads,False,[],False,NaN,/r/legaladvice/comments/ggj9vg/neighbors_encro...,all_ads,False,https://www.reddit.com/r/legaladvice/comments/...,1237033,1.589044e+09,0,NaN,False,NaN
2,NaN,legaladvice,"Hi, I am a worker who was temporarily furlough...",t2_55j1c6ak,False,NaN,0,False,[MN] Would I still receive unemployment benefi...,[],r/legaladvice,False,6,NaN,0,True,t3_ggj838,False,dark,1.0,NaN,public,1,0,{},NaN,False,[],NaN,False,False,NaN,{},NaN,False,1,NaN,False,NaN,False,NaN,[],{},NaN,True,NaN,1.589073e+09,text,6,NaN,NaN,text,self.legaladvice,False,"&lt;!-- SC_OFF --&gt;&lt;div class=""md""&gt;&lt...",NaN,NaN,NaN,NaN,False,True,False,False,False,[],[],False,False,False,False,NaN,[],False,NaN,NaN,NaN,t5_2rawz,NaN,NaN,NaN,ggj838,True,NaN,AlternatorDrounds,NaN,1,True,all_ads,False,[],False,NaN,/r/legaladvice/comments/ggj838/mn_would_i_stil...,all_ads,False,https://www.reddit.com/r/legaladvice/comments/...,1237033,1.589044e+09,0,NaN,False,NaN
3,NaN,legaladvice,I live in Belgium with my 2 daughters and my p...,t2_5ug9l5mp,False,NaN,0,False,My baby’s dad is kicking me out of the house b...,[],r/legaladvice,False,6,NaN,0,True,t3_ggj52z,False,dark,1.0,NaN,public,1,0,{},NaN,False,[],NaN,False,False,NaN,{},NaN,False,1,NaN,False,NaN,False,NaN,[],{},NaN,True,NaN,1.589073e+09,tex

In [26]:
# Ensuring shape of df from CSV is same as above.

legaladvice_from_csv.shape

(1097, 102)

In [27]:
# Checking to ensure no. of unique posts in CSV is same as above.

len(legaladvice_from_csv['name'].unique())

998

**Checking posts scraped from subreddit *r/relationship_advice***

Number of posts in *all_posts* dictionary must be same as in *relationshipadvice_raw_posts.csv*

In [19]:
len(all_posts['relationshipadvice'])

1098

In [20]:
# Converting to dataframe

relationshipadvice_df = pd.DataFrame(all_posts['relationshipadvice'])

In [21]:
relationshipadvice_df.head()

,approved_at_utc,subreddit,selftext,author_fullname,saved,mod_reason_title,gilded,clicked,title,link_flair_richtext,subreddit_name_prefixed,hidden,pwls,link_flair_css_class,downs,hide_score,name,quarantine,link_flair_text_color,upvote_ratio,author_flair_background_color,subreddit_type,ups,total_awards_received,media_embed,author_flair_template_id,is_original_content,user_reports,secure_media,is_reddit_media_domain,is_meta,category,secure_media_embed,link_flair_text,can_mod_post,score,approved_by,author_premium,thumbnail,edited,author_flair_css_class,author_flair_richtext,gildings,content_categories,is_self,mod_note,created,link_flair_type,wls,removed_by_category,banned_by,author_flair_type,domain,allow_live_comments,selftext_html,likes,suggested_sort,banned_at_utc,view_count,archived,no_follow,is_crosspostable,pinned,over_18,all_awardings,awarders,media_only,can_gild,spoiler,locked,author_flair_text,treatment_tags,visited,removed_by,num_reports,distinguished,subreddit_id,mod_reason_by,removal_reason,link_flair_background_color,id,is_robot_indexable,report_reasons,author,discussion_type,num_comments,send_replies,whitelist_status,contest_mode,mod_reports,author_patreon_flair,author_flair_text_color,permalink,parent_whitelist_status,stickied,url,subreddit_subscribers,created_utc,num_crossposts,media,is_video
0,None,relationship_advice,We been dating for almost 2 years now and he s...,t2_54fr010z,False,None,0,False,Should I be upset if my boyfriend doesn’t reme...,[],r/relationship_advice,False,6,None,0,True,t3_ggjd4u,False,dark,1.00,None,public,1,0,{},None,False,[],None,False,False,None,{},None,False,1,None,False,,False,None,[],{},None,True,None,1.589073e+09,text,6,None,None,text,self.relationship_advice,False,"&lt;!-- SC_OFF --&gt;&lt;div class=""md""&gt;&lt...",None,None,None,None,False,True,False,False,False,[],[],False,False,False,False,None,[],False,None,None,None,t5_2r0cn,None,None,,ggjd4u,True,None,lalaxx3,None,0,True,all_ads,False,[],False,None,/r/relationship_advice/comments/ggjd4u/should_...,all_ads,False,https://www.reddit.com/r/relationship_advice/c...,3026114,1.589044e+09,0,None,False
1,None,relationship_advice,I'll split my situation into parts so its easi...,t2_k6tv9,False,None,0,False,What advice could you all give me ?,[],r/relationship_advice,False,6,None,0,True,t3_ggjc7u,False,dark,0.99,None,public,1,0,{},None,False,[],None,False,False,None,{},None,False,1,None,False,,False,None,[],{},None,True,None,1.589073e+09,text,6,None,None,text,self.relationship_advice,False,"&lt;!-- SC_OFF --&gt;&lt;div class=""md""&gt;&lt...",None,None,None,None,False,True,False,False,False,[],[],False,False,False,False,None,[],False,None,None,None,t5_2r0cn,None,None,,ggjc7u,True,None,nicomoli12,None,0,True,all_ads,False,[],False,None,/r/relationship_advice/comments/ggjc7u/what_ad...,all_ads,False,https://www.reddit.com/r/relationship_advice/c...,3026114,1.589044e+09,0,None,False
2,None,relationship_advice,So this is going to be long. I'm not sure if t...,t2_1aasy0iv,False,None,0,False,Confused. Posting here for advice on my relati...,[],r/relationship_advice,False,6,None,0,True,t3_ggjc0o,False,dark,1.00,None,public,1,0,{},None,False,[],None,False,False,None,{},None,False,1,None,False,,False,None,[],{},None,True,None,1.589073e+09,text,6,None,None,text,self.relationship_advice,False,"&lt;!-- SC_OFF --&gt;&lt;div class=""md""&gt;&lt...",None,None,None,None,False,True,False,False,False,[],[],False,False,False,False,None,[],False,None,None,None,t5_2r0cn,None,None,,ggjc0o,True,None,SuKh22,None,0,True,all_ads,False,[],False,None,/r/relationship_advice/comments/ggjc0o/confuse...,all_ads,False,https://www.reddit.com/r/relationship_advice/c...,3026114,1.589044e+09,0,None,False
3,None,relationship_advice,Background: I’m usually not one to look for ad...,t2_5kww2hvk,False,None,0,False,My (26F) Father Sold My Boyfriend (31M) a Brok...,[],r/relationship_advice,False,6,None,0,True,t3_ggjbue,False,dark,1.00,None,public,1,0,{},None,False,[],None,False,

In [22]:
relationshipadvice_df.shape

(1098, 101)

In [32]:
# Checking for unique posts in total scraped posts.

len(relationshipadvice_df['name'].unique())

1002

In [28]:
# Reading the CSV file as dataframe to ensure all data got saved correctly.

relationshipadvice_from_csv = pd.read_csv('../data/relationshipadvice_raw_posts.csv')

In [29]:
relationshipadvice_from_csv.head()

,approved_at_utc,subreddit,selftext,author_fullname,saved,mod_reason_title,gilded,clicked,title,link_flair_richtext,subreddit_name_prefixed,hidden,pwls,link_flair_css_class,downs,hide_score,name,quarantine,link_flair_text_color,upvote_ratio,author_flair_background_color,subreddit_type,ups,total_awards_received,media_embed,author_flair_template_id,is_original_content,user_reports,secure_media,is_reddit_media_domain,is_meta,category,secure_media_embed,link_flair_text,can_mod_post,score,approved_by,author_premium,thumbnail,edited,author_flair_css_class,author_flair_richtext,gildings,content_categories,is_self,mod_note,created,link_flair_type,wls,removed_by_category,banned_by,author_flair_type,domain,allow_live_comments,selftext_html,likes,suggested_sort,banned_at_utc,view_count,archived,no_follow,is_crosspostable,pinned,over_18,all_awardings,awarders,media_only,can_gild,spoiler,locked,author_flair_text,treatment_tags,visited,removed_by,num_reports,distinguished,subreddit_id,mod_reason_by,removal_reason,link_flair_background_color,id,is_robot_indexable,report_reasons,author,discussion_type,num_comments,send_replies,whitelist_status,contest_mode,mod_reports,author_patreon_flair,author_flair_text_color,permalink,parent_whitelist_status,stickied,url,subreddit_subscribers,created_utc,num_crossposts,media,is_video
0,NaN,relationship_advice,We been dating for almost 2 years now and he s...,t2_54fr010z,False,NaN,0,False,Should I be upset if my boyfriend doesn’t reme...,[],r/relationship_advice,False,6,NaN,0,True,t3_ggjd4u,False,dark,1.00,NaN,public,1,0,{},NaN,False,[],NaN,False,False,NaN,{},NaN,False,1,NaN,False,NaN,False,NaN,[],{},NaN,True,NaN,1.589073e+09,text,6,NaN,NaN,text,self.relationship_advice,False,"&lt;!-- SC_OFF --&gt;&lt;div class=""md""&gt;&lt...",NaN,NaN,NaN,NaN,False,True,False,False,False,[],[],False,False,False,False,NaN,[],False,NaN,NaN,NaN,t5_2r0cn,NaN,NaN,NaN,ggjd4u,True,NaN,lalaxx3,NaN,0,True,all_ads,False,[],False,NaN,/r/relationship_advice/comments/ggjd4u/should_...,all_ads,False,https://www.reddit.com/r/relationship_advice/c...,3026114,1.589044e+09,0,NaN,False
1,NaN,relationship_advice,I'll split my situation into parts so its easi...,t2_k6tv9,False,NaN,0,False,What advice could you all give me ?,[],r/relationship_advice,False,6,NaN,0,True,t3_ggjc7u,False,dark,0.99,NaN,public,1,0,{},NaN,False,[],NaN,False,False,NaN,{},NaN,False,1,NaN,False,NaN,False,NaN,[],{},NaN,True,NaN,1.589073e+09,text,6,NaN,NaN,text,self.relationship_advice,False,"&lt;!-- SC_OFF --&gt;&lt;div class=""md""&gt;&lt...",NaN,NaN,NaN,NaN,False,True,False,False,False,[],[],False,False,False,False,NaN,[],False,NaN,NaN,NaN,t5_2r0cn,NaN,NaN,NaN,ggjc7u,True,NaN,nicomoli12,NaN,0,True,all_ads,False,[],False,NaN,/r/relationship_advice/comments/ggjc7u/what_ad...,all_ads,False,https://www.reddit.com/r/relationship_advice/c...,3026114,1.589044e+09,0,NaN,False
2,NaN,relationship_advice,So this is going to be long. I'm not sure if t...,t2_1aasy0iv,False,NaN,0,False,Confused. Posting here for advice on my relati...,[],r/relationship_advice,False,6,NaN,0,True,t3_ggjc0o,False,dark,1.00,NaN,public,1,0,{},NaN,False,[],NaN,False,False,NaN,{},NaN,False,1,NaN,False,NaN,False,NaN,[],{},NaN,True,NaN,1.589073e+09,text,6,NaN,NaN,text,self.relationship_advice,False,"&lt;!-- SC_OFF --&gt;&lt;div class=""md""&gt;&lt...",NaN,NaN,NaN,NaN,False,True,False,False,False,[],[],False,False,False,False,NaN,[],False,NaN,NaN,NaN,t5_2r0cn,NaN,NaN,NaN,ggjc0o,True,NaN,SuKh22,NaN,0,True,all_ads,False,[],False,NaN,/r/relationship_advice/comments/ggjc0o/confuse...,all_ads,False,https://www.reddit.com/r/relationship_advice/c...,3026114,1.589044e+09,0,NaN,False
3,NaN,relationship_advice,Background: I’m usually not one to look for ad...,t2_5kww2hvk,False,NaN,0,False,My (26F) Father Sold My Boyfriend (31M) a Brok...,[],r/relationship_advice,False,6,NaN,0,True,t3_ggjbue,False,dark,1.00,NaN,public,1,0,{},NaN,False,[],NaN,False,False,NaN,{},NaN,False,1,NaN,False,NaN,False,NaN,[],{},NaN,True,NaN,1.58

In [30]:
# Ensuring shape of df from CSV is same as above.

relationshipadvice_from_csv.shape

(1098, 101)

In [31]:
# Checking to ensure no. of unique posts in CSV is same as above.

len(relationshipadvice_from_csv['name'].unique())

1002